## Gymnasium - An API standard for reinforcement learning with a diverse collection of reference environments

Documents: https://gymnasium.farama.org/introduction/train_agent/

Gymnasium is a project that provides an API (application programming interface) for all single agent reinforcement learning environments, with implementations of common environments: cartpole, pendulum, mountain-car, mujoco, atari, and more.

# Stable Baselines3 - Training, Saving and Loading

Github Repo: [https://github.com/DLR-RM/stable-baselines3](https://github.com/DLR-RM/stable-baselines3)


[RL Baselines3 Zoo](https://github.com/DLR-RM/rl-baselines3-zoo) is a training framework for Reinforcement Learning (RL), using Stable Baselines3.

It provides scripts for training, evaluating agents, tuning hyperparameters, plotting results and recording videos.

Documentation is available online: [https://stable-baselines3.readthedocs.io/](https://stable-baselines3.readthedocs.io/)

Examples with Collab Code: [https://stable-baselines3.readthedocs.io/en/master/guide/examples.html](https://stable-baselines3.readthedocs.io/en/master/guide/examples.html)

## Install Dependencies and Stable Baselines Using Pip


```
pip install stable-baselines3[extra]
```

In [ ]:
# for autoformatting
# %load_ext jupyter_black


In [ ]:
!pip install box2d-py
!pip install "stable-baselines3[extra]>=2.0.0a4"


## Import policy, RL agent, ...

In [4]:
import gymnasium as gym
import numpy as np

from stable_baselines3 import DQN
from stable_baselines3.common.callbacks import CheckpointCallback


## Create the Gym env and instantiate the agent

For this example, we will use Lunar Lander environment.

"Landing outside landing pad is possible. Fuel is infinite, so an agent can learn to fly and then land on its first attempt. Four discrete actions available: do nothing, fire left orientation engine, fire main engine, fire right orientation engine. "

Lunar Lander environment: [https://gymnasium.farama.org/environments/box2d/lunar_lander/](https://gymnasium.farama.org/environments/box2d/lunar_lander/)

![Lunar Lander](https://cdn-images-1.medium.com/max/960/1*f4VZPKOI0PYNWiwt0la0Rg.gif)


We chose the MlpPolicy because input of Lunar Lander is a feature vector, not images.

The type of action to use (discrete/continuous) will be automatically deduced from the environment action space



In [5]:
model = DQN(
    "MlpPolicy",
    "LunarLander-v3",
    verbose=1,
    exploration_final_eps=0.1,
    target_update_interval=250,
    tensorboard_log="tb_logs",
)

Using cpu device
Creating environment from the given name 'LunarLander-v3'
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


We load a helper function to evaluate the agent:

In [6]:
from stable_baselines3.common.evaluation import evaluate_policy

Let's evaluate the un-trained agent, this should be a random agent.

In [7]:
# Separate env for evaluation
eval_env = gym.make("LunarLander-v3", render_mode="human")


# Random Agent, before training
mean_reward, std_reward = evaluate_policy(
    model,
    eval_env,
    n_eval_episodes=10,
    deterministic=True,
)

print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

c:\PycharmProjects\gym_jupyter\.venv\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


mean_reward=-597.00 +/- 165.98748966265896


## Train the agent and save it

Warning: this may take a while

In [8]:
# Save every 10000 steps
checkpoint_callback = CheckpointCallback(
    save_path="models/dqn_lunar",
    save_freq=10000,
    name_prefix="dqn_lunar",
)
# Train the agent
model.learn(total_timesteps=int(1e5), callback=checkpoint_callback, tb_log_name="dqn_lunar")
# Save the agent optionally
model.save("dqn_lunar_final")
del model  # delete trained model to demonstrate loading

Logging to tb_logs\dqn_lunar_2
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 80.8     |
|    ep_rew_mean      | -112     |
|    exploration_rate | 0.971    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 1158     |
|    time_elapsed     | 0        |
|    total_timesteps  | 323      |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 4.06     |
|    n_updates        | 55       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 82.2     |
|    ep_rew_mean      | -211     |
|    exploration_rate | 0.941    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 1246     |
|    time_elapsed     | 0        |
|    total_timesteps  | 658      |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 

## Load the trained agent

In [ ]:
model = DQN.load("dqn_lunar_final")

# load a specific checkpoint file


In [8]:
eval_env = gym.make("LunarLander-v3", render_mode="human")

# add render_mode="human" to visualize the results

In [9]:
# Evaluate the trained agent
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=5, deterministic=True)

print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

c:\PycharmProjects\gym_jupyter\.venv\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


mean_reward=-96.72 +/- 23.48040326224806


In [10]:
eval_env.close()